In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from MyDatasetLoader import MyDatasetLoader
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import KFold
import torch
import numpy as np


def reduce_dim_pca(dataset,reduce_dim_to):
    pca = PCA(n_components=reduce_dim_to)
    x= pca.fit_transform(dataset[0].x)
   
    dataset.data.x =torch.tensor(x,dtype=torch.float32)
    return 
dataset=MyDatasetLoader(root='./cora')
#reduce_dim_pca(dataset,300)

train_x= dataset[0].x[dataset[0].train_mask].to('cpu').numpy()
train_y= dataset[0].y[dataset[0].train_mask].to('cpu').numpy()
test_x = dataset[0].x[dataset[0].test_mask].to('cpu').numpy()
test_y = dataset[0].y[dataset[0].test_mask].to('cpu').numpy()




Processing...
Done!


In [21]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
train_x= dataset[0].x[dataset[0].train_mask].to('cpu').numpy()
train_y= dataset[0].y[dataset[0].train_mask].to('cpu').numpy()
test_x = dataset[0].x[dataset[0].test_mask].to('cpu').numpy()
test_y = dataset[0].y[dataset[0].test_mask].to('cpu').numpy()

full_train_x=dataset[0].x.to('cpu').numpy()
full_train_y=dataset[0].y.to('cpu').numpy()


hot_full_train_y= ohe.fit_transform(full_train_y.reshape(-1, 1)).toarray()
print(len(hot_full_train_y))
print(len(full_train_x))


val_x = dataset[0].x[dataset[0].val_mask].to('cpu').numpy()
val_y = dataset[0].y[dataset[0].val_mask].to('cpu').numpy()
hot_train_y = ohe.fit_transform(train_y.reshape(-1, 1)).toarray()
hot_test_y=  ohe.fit_transform(test_y.reshape(-1, 1)).toarray()
hot_val_y=  ohe.fit_transform(val_y.reshape(-1, 1)).toarray()


def convert_from_one_hot(pred_one_hot):
    pred = list()
    for i in range(len(pred_one_hot)):
        pred.append(np.argmax(pred_one_hot[i]))
    return pred



2708
2708


In [16]:
model = Sequential()
model.add(Dense(50, input_dim=len(train_x[0]), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_x, hot_train_y, epochs=50)

Epoch 1/50
51/51 [==============================] - 0s 931us/step - loss: 1.8754 - accuracy: 0.2714
Epoch 2/50
51/51 [==============================] - 0s 931us/step - loss: 1.7507 - accuracy: 0.3138
Epoch 3/50
51/51 [==============================] - 0s 922us/step - loss: 1.6313 - accuracy: 0.3194
Epoch 4/50
51/51 [==============================] - 0s 902us/step - loss: 1.4726 - accuracy: 0.4043
Epoch 5/50
51/51 [==============================] - 0s 902us/step - loss: 1.2946 - accuracy: 0.5298
Epoch 6/50
51/51 [==============================] - 0s 971us/step - loss: 1.1181 - accuracy: 0.6246
Epoch 7/50
51/51 [==============================] - 0s 931us/step - loss: 0.8677 - accuracy: 0.6972
Epoch 8/50
51/51 [==============================] - 0s 951us/step - loss: 0.7566 - accuracy: 0.7262
Epoch 9/50
51/51 [==============================] - 0s 912us/step - loss: 0.6293 - accuracy: 0.7766
Epoch 10/50
51/51 [==============================] - 0s 892us/step - loss: 0.5592 - accuracy: 0.8037

In [17]:

pred=model.predict(test_x)

y_pred=convert_from_one_hot(pred)

print("Accuracy:",metrics.accuracy_score(test_y, y_pred))
print("f1:",metrics.f1_score(test_y, y_pred,average='macro'))
print("confusion matrix:\n",metrics.confusion_matrix(test_y, y_pred))

Accuracy: 0.16236162361623616
f1: 0.039909297052154194
confusion matrix:
 [[  0   0   0   0   0 235   0]
 [  0   0   0   0   0  62   0]
 [  0   0   0   0   0  60   0]
 [  0   0   0   0   0 120   0]
 [  0   0   0   0   0 114   0]
 [  0   0   0   0   0 132   0]
 [  0   0   0   0   0  90   0]]


In [113]:

pred=model.predict(train_x)

y_pred=convert_from_one_hot(pred)

print("Accuracy:",metrics.accuracy_score(train_y, y_pred))
print("f1:",metrics.f1_score(train_y, y_pred,average='macro'))
print("confusion matrix:\n",metrics.confusion_matrix(train_y, y_pred))

Accuracy: 0.9981538461538462
f1: 0.9986674966941893
confusion matrix:
 [[506   0   0   1   0   0   0]
 [  0 107   0   0   0   0   0]
 [  0   0 130   0   0   0   0]
 [  1   0   0 263   0   0   0]
 [  1   0   0   0 193   0   0]
 [  0   0   0   0   0 248   0]
 [  0   0   0   0   0   0 175]]


In [114]:

pred=model.predict(val_x)

y_pred=convert_from_one_hot(pred)

print("Accuracy:",metrics.accuracy_score(val_y, y_pred))
print("f1:",metrics.f1_score(val_y, y_pred,average='macro'))
print("confusion matrix:\n",metrics.confusion_matrix(val_y, y_pred))

Accuracy: 0.14074074074074075
f1: 0.03525046382189239
confusion matrix:
 [[ 0  0  0  0  0 76  0]
 [ 0  0  0  0  0 11  0]
 [ 0  0  0  0  0 27  0]
 [ 0  0  0  0  0 42  0]
 [ 0  0  0  0  0 43  0]
 [ 0  0  0  0  0 38  0]
 [ 0  0  0  0  0 33  0]]


In [25]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
loss_function = sparse_categorical_crossentropy
kfold = KFold(n_splits=3, shuffle=True)
acc_per_fold = []
loss_per_fold = []
models=[]
fold_no=0

#train_x=full_train_x
#train_y=full_train_y
#hot_train_y=hot_full_train_y

for train, test in kfold.split(train_x, hot_train_y):
   
    
    model = Sequential()
    model.add(Dense(50, input_dim=len(train_x[0]), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7, activation='softmax'))

    model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(train_x[train], train_y[train],
              batch_size=100,
              epochs=50,
             )
    scores = model.evaluate(train_x[test], train_y[test], verbose=0)
    models.append(model)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    pred=model.predict(train_x[test])
    y_pred=convert_from_one_hot(pred)
    print("Accuracy:",metrics.accuracy_score( train_y[test], y_pred))
    print("f1:",metrics.f1_score( train_y[test], y_pred,average='macro'))
    print("confusion matrix:\n",metrics.confusion_matrix(train_y[test], y_pred))
    
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1





------------------------------------------------------------------------
Training for fold 0 ...
Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 1.9365 - accuracy: 0.2059
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 1.8712 - accuracy: 0.3158
Epoch 3/50
11/11 [==============================] - 0s 2ms/step - loss: 1.7634 - accuracy: 0.3186
Epoch 4/50
11/11 [==============================] - 0s 2ms/step - loss: 1.6374 - accuracy: 0.3176
Epoch 5/50
11/11 [==============================] - 0s 2ms/step - loss: 1.4554 - accuracy: 0.4183
Epoch 6/50
11/11 [==============================] - 0s 2ms/step - loss: 1.2301 - accuracy: 0.5531
Epoch 7/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9670 - accuracy: 0.6676
Epoch 8/50
11/11 [==============================] - 0s 2ms/step - loss: 0.7490 - accuracy: 0.7682
Epoch 9/50
11/11 [==============================] - 0s 2ms/step - loss: 0.5572 - accuracy: 0.8393
Epoch 10/50
11/11 [==

ValueError: Found input variables with inconsistent numbers of samples: [813, 542]

In [23]:

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')



------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 1.344421625137329 - Accuracy: 77.12177038192749%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.6538147926330566 - Accuracy: 71.95571660995483%
------------------------------------------------------------------------
> Fold 3 - Loss: 1.78190016746521 - Accuracy: 73.3826220035553%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 74.15336966514587 (+- 2.1783123225888636)
> Loss: 1.5933788617451985
------------------------------------------------------------------------


In [24]:

for model,fold in zip(models,range(0,len(models))):
    print(f'-------fold number {fold}------')
    pred=model.predict(test_x)
    y_pred=convert_from_one_hot(pred)
    print("Accuracy:",metrics.accuracy_score(test_y, y_pred))
    print("f1:",metrics.f1_score(test_y, y_pred,average='macro'))
    print("confusion matrix:\n",metrics.confusion_matrix(test_y, y_pred))

-------fold number 0------
Accuracy: 0.2890528905289053
f1: 0.06406761177753544
confusion matrix:
 [[235   0   0   0   0   0   0]
 [ 62   0   0   0   0   0   0]
 [ 60   0   0   0   0   0   0]
 [120   0   0   0   0   0   0]
 [114   0   0   0   0   0   0]
 [132   0   0   0   0   0   0]
 [ 90   0   0   0   0   0   0]]
-------fold number 1------
Accuracy: 0.2890528905289053
f1: 0.06406761177753544
confusion matrix:
 [[235   0   0   0   0   0   0]
 [ 62   0   0   0   0   0   0]
 [ 60   0   0   0   0   0   0]
 [120   0   0   0   0   0   0]
 [114   0   0   0   0   0   0]
 [132   0   0   0   0   0   0]
 [ 90   0   0   0   0   0   0]]
-------fold number 2------
Accuracy: 0.2890528905289053
f1: 0.06406761177753544
confusion matrix:
 [[235   0   0   0   0   0   0]
 [ 62   0   0   0   0   0   0]
 [ 60   0   0   0   0   0   0]
 [120   0   0   0   0   0   0]
 [114   0   0   0   0   0   0]
 [132   0   0   0   0   0   0]
 [ 90   0   0   0   0   0   0]]
